# Input of txt-file -> transferring to dataFrame

In [26]:
import numpy as np
import pandas as pd
import os

# Function: (Getting) paths_and_names_files

In [10]:
def paths_and_names_files(file_type):
    '''
    Input: file_type = '.txt' or '.JPG' (string)
    Getting each individual path and name of the file-type.
    '''

    # Define the directory to search for the files - path = path to all our 519 training images and their label-txt.files
    directory_path= '/Users/myself/Pictures/Kestrix Bootcamp Project/Training_Data_Kestrix/obj_Train_data'

    # Get a list of all files in the directory
    all_files = os.listdir(directory_path)

    # Filter the list to include only .jpg files and sort them
    spec_files = sorted([f for f in all_files if f.endswith(file_type)])

    # Filter the list to include only the specified file types and create a dictionary
    # length = 519
    spec_files_dict = {f: os.path.join(directory_path, f) for f in spec_files}


    list_files = []          # path to every single file
    list_file_names = []     # name of every single file
    a = 0

    for key, value in spec_files_dict.items():
    # print(key, value)
        if a < 525:
            list_files.append(value)
            list_file_names.append(key)
        a += 1

    return list_files, list_file_names

In [30]:
file_type_txt = '.txt'
file_type_img = '.JPG'

list_txts, list_txt_names = paths_and_names_files(file_type_txt)
list_imgs, list_img_names = paths_and_names_files(file_type_img)

len(list_txts), len(list_imgs)
# works!

(519, 519)

# Function: Preparing for-loops

In [31]:
def preparing_for_loops():

    # Create an empty DataFrame with the specified columns
    columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
    data_frame = pd.DataFrame(columns=columns)
    data_frame

    # Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
    dict_df_comps = {}

    for i in range (0, 48):
        dict_df_comps[i] = f'df_comp_{i}'

    return data_frame, dict_df_comps

# Whole Pipeline

In [60]:
file_type_txt = '.txt'
file_type_img = '.JPG'

list_txts, list_txt_names = paths_and_names_files(file_type_txt)
list_imgs, list_img_names = paths_and_names_files(file_type_img)

for ele in range(0, len(list_txts)):

    path_annotations = list_txts[ele]

    # Read the file with a space delimiter and assign the column names
    column_names = ["class", "x_center", "y_center", "width", "height"]
    df = pd.read_csv(path_annotations, names=column_names, delimiter=' ')
    # print(df)

    # Running necessary functions
    coordinates_dict = luc_coordinates()

    slicing_dict = slicing_dictionary(coordinates_dict)

    image_bb = getting_bounding_boxes(df)

    data_frame, dict_df_comps = preparing_for_loops()

    # For Loop for each compartment (compartment-level)
    for key, value in slicing_dict.items():
        #print(key)
        #print(value)
        compartment_num = key
        # print(f'compartment', key)
        dict_df_comps[key] = data_frame.copy()

        # For Loop for each bounding box
        for k, v in image_bb.items():

            # calling function: checking_bounding_box (Output = True or False)
            x_min = v[0]
            y_min = v[1]
            x_max = v[2]
            y_max = v[3]
            class_object = v[4]
            #print(x_min)
            #print(y_min)
            #print(x_max)
            #print(y_max)
            if checking_bounding_box(compartment_num, x_min, y_min, x_max, y_max):
                # print('True')
                #calling the function: checking_for_9_cases
                x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases(x_min, y_min, x_max, y_max)
                # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy(compartment_num, x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

                #Adding those to a dataframe with the original class
                list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
                dict_df_comps[key].loc[len(dict_df_comps[key])] = list_ready
                # else:
                # print('False')
                # continue or cut the else
        # print(dict_df_comps.keys())

    for comp_num, _ in dict_df_comps.items():
        # print(comp_num, value)
        # Save DataFrame to .txt file
        dict_df_comps[comp_num].to_csv(f'/Users/myself/Pictures/Kestrix Bootcamp Project/testing_saving_txts/{list_img_names[ele]}_{comp_num}.txt', sep=' ',index=False, index_label = False, header=False)

    #Output is the dict_df_comps


17
500 1140
1000 1640
1000 1640
18
1000 1640
1000 1640
1000 1640
1000 1640
31
3500 4140
3500 4140
1500 2140
36
2000 2640
2000 2640
2000 2640
2000 2640
36
2000 2640
2000 2640
2000 2640
37
2500 3140
2000 2640
2000 2640
37
2500 3140
2000 2640
2000 2640
37
2500 3140
2500 3140
2000 2640
2000 2640
38
3000 3640
3000 3640
2000 2640
2000 2640
38
3000 3640
3000 3640
2000 2640
2000 2640
39
3500 4140
2000 2640
2000 2640
39
3500 4140
3500 4140
2000 2640
2000 2640
44
2000 2640
2000 2640
2500 3140
44
2000 2640
2500 3140
45
2500 3140
2500 3140
45
2500 3140
2500 3140
45
2500 3140
2500 3140
2500 3140
46
3000 3640
3000 3640
2500 3140
46
3000 3640
3000 3640
2500 3140
47
3500 4140
2500 3140
36
2000 2640
2000 2640
2000 2640
36
2000 2640
2000 2640
37
2500 3140
2000 2640
37
2500 3140
2000 2640
37
2500 3140
2500 3140
2000 2640
38
3000 3640
3000 3640
2000 2640
38
3000 3640
3000 3640
2000 2640
39
3500 4140
2000 2640
39
3500 4140
3500 4140
2000 2640
44
2000 2640
2000 2640
2500 3140
2500 3140
44
2000 2640
2500 314

# Pipeline-Function

In [75]:
def pipeline_function():
    file_type_txt = '.txt'
    file_type_img = '.JPG'

    list_txts, _ = paths_and_names_files(file_type_txt)
    _, list_img_names = paths_and_names_files(file_type_img)

    for ele in range(0, len(list_txts)):

        path_annotations = list_txts[ele]

        # Read the file with a space delimiter and assign the column names
        column_names = ["class", "x_center", "y_center", "width", "height"]
        df = pd.read_csv(path_annotations, names=column_names, delimiter=' ')
        #  print(df)

        # Running necessary functions
        coordinates_dict = luc_coordinates()

        slicing_dict = slicing_dictionary(coordinates_dict)

        image_bb = getting_bounding_boxes(df)

        data_frame, dict_df_comps = preparing_for_loops()

        # For Loop for each compartment (compartment-level)
        for key, value in slicing_dict.items():
            # print(key)
            # print(value)
            compartment_num = key
            # print(f'compartment', key)
            dict_df_comps[key] = data_frame.copy()

            # For Loop for each bounding box
            for k, v in image_bb.items():

                # calling function: checking_bounding_box (Output = True or False)
                # print(compartment_num, image_bb)
                x_min = v[0]
                y_min = v[1]
                x_max = v[2]
                y_max = v[3]
                class_object = v[4]
                # print(x_min)
                # print(y_min)
                # print(x_max)
                # print(y_max)
                if checking_bounding_box(compartment_num, x_min, y_min, x_max, y_max):
                    # print('True')
                    #calling the function: checking_for_9_cases
                    # print(compartment_num)
                    x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases(compartment_num, x_min, y_min, x_max, y_max)
                    # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                    abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy(compartment_num, x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                    # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

                    #Adding those to a dataframe with the original class
                    list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
                    dict_df_comps[key].loc[len(dict_df_comps[key])] = list_ready
                    # else:
                    # print('False')
                    # continue or cut the else
            # print(dict_df_comps.keys())

        for comp_num, _ in dict_df_comps.items():
            # print(comp_num, _)
            # Save DataFrame to .txt file
            dict_df_comps[comp_num].to_csv(f'/Users/myself/Pictures/Kestrix Bootcamp Project/testing_saving_txts/{list_img_names[ele]}_{comp_num}.txt', sep=' ',index=False, index_label = False, header=False)

        #Output is the dict_df_comps
        # return print('Done - hope it worked')


In [76]:
pipeline_function()

# Function: **luc_coordinates**

In [32]:
def luc_coordinates():
    ''' A function that takes as an input (given in the function already)
    width (of the image) = 4000
    height (of the image) = 3000
    width_comp = number of compartments for the width (e.g. 8)
    height_comp = number of compartments for the height (e.g. 6)
    comp_size = size of each compartment (e.g. 640)
    and returns the coordinates of the left-upper-corner (luc) of
    each compartment
    '''

    # Given inputs
    width = 4000
    height = 3000
    num_width_comp = 8
    num_height_comp = 6
    comp_size = 640

    # Step size is the size of the compartments unique pixels horizontally and vertically
    step_size_horiz = width / num_width_comp
    step_size_vert = height / num_height_comp

    #creating a list with all the x_coordinates for the compartments (8)
    x_coordinates = [0]
    [x_coordinates.append(int(ele * step_size_horiz)) for ele in range(1, num_width_comp)]

    # creating a list with all the y_coordinates for the compartments (6)
    y_coordinates = [0]
    [y_coordinates.append(int(ele * step_size_vert)) for ele in range(1, num_height_comp)]


    # Creating a dictionary with the coordinates
    coordinates_dict = {}
    b = 0 # no inherent important meaning

    for ele in range (0, num_height_comp):
        for i in range (0, num_width_comp):
            a = [x_coordinates[i], y_coordinates[ele]]
            b += 1
            coordinates_dict[f'cor_{b}'] = a

    return coordinates_dict

# Function: **slicing_dictionary**

In [33]:
def slicing_dictionary(coordinates_dict):

    # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
    # coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

    slize_size = 640

    # Turning the coordinates into a list
    coordinates_list = [value for key, value in coordinates_dict.items()]

    # creating a for loop for getting the correct slizing integers and putting them into a dictionary
    slicing_dict = {}

    for i in range (0, 48):
        slice_1, slice_2 = coordinates_list[i][0], coordinates_list[i][1]
        slice_1_a = slice_1
        slice_1_b = slice_1_a + slize_size
        slice_2_a = slice_2
        slice_2_b = slice_2_a + slize_size
        slicing_dict[i] = [slice_1_a, slice_1_b, slice_2_a, slice_2_b]

    return slicing_dict

# Function: getting_bounding_boxes

In [34]:
def getting_bounding_boxes(df):
    '''
    Requirement: Having access to the annotations for one image in
    a pd.DataFrame
    Inputs:
    df = DataFrame of one compartments' bounding boxes
    Outputs:
    image_bb = Dictionary with Key: enumerated bounding box Value: list of
    the 5 values of each bounding box (x_min, y_min, x_max, y_max, class_object)
    '''

    width_img = 4000
    height_img = 3000

    # Getting a dictionary with key: enumerated bounding box and value: list of the 5 values
    # of each bounding box (x_min, y_min, x_max, y_max, class_object)
    image_bb = {}

    for bb in range(0, df.shape[0]):

        # getting the x_center, y_center, width, height from the first bounding box
        x_center = df.iloc[bb]['x_center']                  # = 0.785663
        y_center = df.iloc[bb]['y_center']                  # = 0.776272
        first_width = df.iloc[bb]['width']                  # = 0.05003
        first_height = df.iloc[bb]['height']                # = 0.109817
        class_object = int(df.iloc[bb]['class'])            # = 0 ('car')

        #MAYBE splitting what's above and creating a for-loop for all the annotations

        # Transferring to the actual width and height of the image
        x_center = round(x_center * width_img)            # = 3143
        y_center = round(y_center * height_img)           # = 2329
        first_width = round(first_width * width_img)        # = 200
        first_height = round(first_height * height_img)     # = 329

        # Calculating the 4 cornerpoints, for which we need 4 numbers: x_min, x_max, y_min, y_max
        # (Adding 70 for each because of the padding of zeros around the original image)
        x_min = int(x_center - (first_width / 2)) + 70  # left = 3043
        x_max = int(x_center + (first_width / 2)) + 70 # right = 3243
        y_min = int(y_center - (first_height / 2))+ 70 # top = 2164
        y_max = int(y_center + (first_height / 2))+ 70 # bottom = 2493

        image_bb[bb] = [x_min, y_min, x_max, y_max, class_object]

    return image_bb


# Function: checking_bounding_box



In [35]:
def checking_bounding_box(compartment_num, x_min, y_min, x_max, y_max):
  '''
  Input to this function is the number of the compartment for which it
  should be tested whether the bounding box is inside or not
  And the output of function: getting_bounding_boxes
  With this we want to take one row of the txt.file (pd.DataFrame)
  that represents one bounding box in the original image
  Output of the function:
  True if some part of the bounding box lies in the compartment
  False if the bounding box doesn't lie in the compartment!
  '''

  # If one of x_min, x_max AND one of y_min, y_max in the compartment -> True else False
  if ((x_min in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1)
    or x_max in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] +1))
    and
     ((y_min in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1)
    or y_max in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1)))):
    tmp_var = True
  else:
    tmp_var = False


#[slicing_dict[0][0]:slicing_dict[0][1],
#slicing_dict[0][2]:slicing_dict[0][3]
  return tmp_var

# Function: checking_for_9_cases

In [61]:
def checking_for_9_cases(compartment_num, x_min, y_min, x_max, y_max):
    '''
    ONLY ACTIVATE IF function: checking_bounding_box == TRUE
    Because then we know: Some part of that bounding box lies in
    this compartment!
    We want to
    '''

    # Checking for the next cases (9!)

    # Setting x_min to its value if it is in the range - otherwise setting it to be at the edge (x_min = 0!)
    if x_min in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1):
        # print(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1])
        x_min_tmp = x_min
    else:
        x_min_tmp = slicing_dict[compartment_num][0]

    # Setting x_max to its value if it is in the range - otherwise setting it to be at the edge (x_max = 1!)
    if x_max in range(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1] + 1):
        # print(slicing_dict[compartment_num][0], slicing_dict[compartment_num][1])
        x_max_tmp = x_max
    else:
        x_max_tmp = slicing_dict[compartment_num][1]

    # Doing the same for y_min and y_max!
    if y_min in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1):
        # print(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3])
        y_min_tmp = y_min
    else:
        y_min_tmp = slicing_dict[compartment_num][2]

    if y_max in range(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3] + 1):
        # print(slicing_dict[compartment_num][2], slicing_dict[compartment_num][3])
        y_max_tmp = y_max
    else:
        y_max_tmp = slicing_dict[compartment_num][3]

    # Returning the 4 coordinates in the XYXY class - format
    return x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp

In [59]:
slicing_dict

{0: [0, 640, 0, 640],
 1: [500, 1140, 0, 640],
 2: [1000, 1640, 0, 640],
 3: [1500, 2140, 0, 640],
 4: [2000, 2640, 0, 640],
 5: [2500, 3140, 0, 640],
 6: [3000, 3640, 0, 640],
 7: [3500, 4140, 0, 640],
 8: [0, 640, 500, 1140],
 9: [500, 1140, 500, 1140],
 10: [1000, 1640, 500, 1140],
 11: [1500, 2140, 500, 1140],
 12: [2000, 2640, 500, 1140],
 13: [2500, 3140, 500, 1140],
 14: [3000, 3640, 500, 1140],
 15: [3500, 4140, 500, 1140],
 16: [0, 640, 1000, 1640],
 17: [500, 1140, 1000, 1640],
 18: [1000, 1640, 1000, 1640],
 19: [1500, 2140, 1000, 1640],
 20: [2000, 2640, 1000, 1640],
 21: [2500, 3140, 1000, 1640],
 22: [3000, 3640, 1000, 1640],
 23: [3500, 4140, 1000, 1640],
 24: [0, 640, 1500, 2140],
 25: [500, 1140, 1500, 2140],
 26: [1000, 1640, 1500, 2140],
 27: [1500, 2140, 1500, 2140],
 28: [2000, 2640, 1500, 2140],
 29: [2500, 3140, 1500, 2140],
 30: [3000, 3640, 1500, 2140],
 31: [3500, 4140, 1500, 2140],
 32: [0, 640, 2000, 2640],
 33: [500, 1140, 2000, 2640],
 34: [1000, 1640, 200

In [58]:
x_min = 1045
y_min = 1164
x_max = 1233
y_max = 1302
compartment_num = 17

x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases(x_min, y_min, x_max, y_max)
x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp

500 1140
1000 1640
1000 1640


(1045, 1164, 1140, 1302)

# Function: to_absolute_coordinates_xyxy


In [37]:
def to_absolute_coordinates_xyxy(compartment_num, x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp):
  '''
  Calculates the absolute coordinates of a bounding box
  for a single compartment
  Output: absolute coordinates in form (xyxy = x_min, y_min, x_max, y_max)
  '''
  abs_x_min = x_min_tmp - slicing_dict[compartment_num][0]
  # print(slicing_dict[compartment_num][0])
  abs_x_max = x_max_tmp - slicing_dict[compartment_num][0]
  abs_y_min = y_min_tmp - slicing_dict[compartment_num][2]
  # print(slicing_dict[compartment_num][2])
  abs_y_max = y_max_tmp - slicing_dict[compartment_num][2]

  return abs_x_min, abs_y_min, abs_x_max, abs_y_max

# Function: to_absolute_coordinates_xyxy_with_threshold

In [ ]:
def to_absolute_coordinates_xyxy_with_threshold(compartment_num, x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp):
  '''
  Calculates the absolute coordinates of a bounding box
  for a single compartment depending on how much of this bounding box lies in
  the compartment
  Output: absolute coordinates in form (xyxy = x_min, y_min, x_max, y_max)
  '''
  abs_x_min = x_min_tmp - slicing_dict[compartment_num][0]
  # print(slicing_dict[compartment_num][0])
  abs_x_max = x_max_tmp - slicing_dict[compartment_num][0]
  abs_y_min = y_min_tmp - slicing_dict[compartment_num][2]
  # print(slicing_dict[compartment_num][2])
  abs_y_max = y_max_tmp - slicing_dict[compartment_num][2]



  return abs_x_min, abs_y_min, abs_x_max, abs_y_max

In [39]:
threshold = 0.75 # example

surface_org = (x_max - x_min) * (y_max - y_min) # 500
surface_cut = (abs_x_max - abs_x_min) * (y_max - y_min) = # 300

if (surface_cut/surface_org) > threshold:
    ...


{0: [0, 640, 0, 640],
 1: [500, 1140, 0, 640],
 2: [1000, 1640, 0, 640],
 3: [1500, 2140, 0, 640],
 4: [2000, 2640, 0, 640],
 5: [2500, 3140, 0, 640],
 6: [3000, 3640, 0, 640],
 7: [3500, 4140, 0, 640],
 8: [0, 640, 500, 1140],
 9: [500, 1140, 500, 1140],
 10: [1000, 1640, 500, 1140],
 11: [1500, 2140, 500, 1140],
 12: [2000, 2640, 500, 1140],
 13: [2500, 3140, 500, 1140],
 14: [3000, 3640, 500, 1140],
 15: [3500, 4140, 500, 1140],
 16: [0, 640, 1000, 1640],
 17: [500, 1140, 1000, 1640],
 18: [1000, 1640, 1000, 1640],
 19: [1500, 2140, 1000, 1640],
 20: [2000, 2640, 1000, 1640],
 21: [2500, 3140, 1000, 1640],
 22: [3000, 3640, 1000, 1640],
 23: [3500, 4140, 1000, 1640],
 24: [0, 640, 1500, 2140],
 25: [500, 1140, 1500, 2140],
 26: [1000, 1640, 1500, 2140],
 27: [1500, 2140, 1500, 2140],
 28: [2000, 2640, 1500, 2140],
 29: [2500, 3140, 1500, 2140],
 30: [3000, 3640, 1500, 2140],
 31: [3500, 4140, 1500, 2140],
 32: [0, 640, 2000, 2640],
 33: [500, 1140, 2000, 2640],
 34: [1000, 1640, 200

# Pipeline (1)

**Input**

**1. Pipe**

**2. Pipe**

**1. und 2. Pipe executed**

**Global Run (Function doesn't work)**

In [75]:
# For Loop for each compartment (compartment-level)
for key, value in slicing_dict.items():
  #print(key)
  #print(value)
  compartment_num = key
  print(f'compartment', key)
  dict_df_comps[key] = data_frame.copy()

  # For Loop for each bounding box
  for k, v in cp_bb.items():

    # calling function: checking_bounding_box (Output = True or False)
    x_min = v[0]
    y_min = v[1]
    x_max = v[2]
    y_max = v[3]
    class_object = v[4]
    #print(x_min)
    #print(y_min)
    #print(x_max)
    #print(y_max)
    if checking_bounding_box(compartment_num):
      print('True')
      #calling the function: checking_for_9_cases
      x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
      # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
      abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
      # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

      #Adding those to a dataframe with the original class
      list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
      dict_df_comps[key].loc[len(dict_df_comps[key])] = list_ready
    # else:
      # print('False')
      # continue or cut the else
  print(dict_df_comps[key])

  #Output is the dict_df_comps

compartment 0
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 1
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 2
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 3
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 4
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 5
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 6
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 7
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 8
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 9
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 10
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
compartment 11
Empty DataFrame
Columns: [class, x_min, y_min, x_

**Getting the desired Output**

(= A dictionary with Key = number of compartment, Value = DataFrame with annotations for this compartment)

In [30]:
dict_df_comps[38]

,class,x_min,y_min,x_max,y_max
0,0,594,0,640,139


**Constructing the exporting-function with a for-loop**

In [77]:
for key, _ in dict_df_comps.items():
    print(key, value)
    # Save DataFrame to .txt file
    dict_df_comps[key].to_csv(f'/Users/myself/Kestrix_Project/data/tensor_ex/comp_{key}.txt', sep=' ',index=False, index_label = False, header=False)

dict_df_comps

0 [3500, 4140, 2500, 3140]
1 [3500, 4140, 2500, 3140]
2 [3500, 4140, 2500, 3140]
3 [3500, 4140, 2500, 3140]
4 [3500, 4140, 2500, 3140]
5 [3500, 4140, 2500, 3140]
6 [3500, 4140, 2500, 3140]
7 [3500, 4140, 2500, 3140]
8 [3500, 4140, 2500, 3140]
9 [3500, 4140, 2500, 3140]
10 [3500, 4140, 2500, 3140]
11 [3500, 4140, 2500, 3140]
12 [3500, 4140, 2500, 3140]
13 [3500, 4140, 2500, 3140]
14 [3500, 4140, 2500, 3140]
15 [3500, 4140, 2500, 3140]
16 [3500, 4140, 2500, 3140]
17 [3500, 4140, 2500, 3140]
18 [3500, 4140, 2500, 3140]
19 [3500, 4140, 2500, 3140]
20 [3500, 4140, 2500, 3140]
21 [3500, 4140, 2500, 3140]
22 [3500, 4140, 2500, 3140]
23 [3500, 4140, 2500, 3140]
24 [3500, 4140, 2500, 3140]
25 [3500, 4140, 2500, 3140]
26 [3500, 4140, 2500, 3140]
27 [3500, 4140, 2500, 3140]
28 [3500, 4140, 2500, 3140]
29 [3500, 4140, 2500, 3140]
30 [3500, 4140, 2500, 3140]
31 [3500, 4140, 2500, 3140]
32 [3500, 4140, 2500, 3140]
33 [3500, 4140, 2500, 3140]
34 [3500, 4140, 2500, 3140]
35 [3500, 4140, 2500, 3140]
36

{0: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 1: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 2: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 3: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 4: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 5: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 6: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 7: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 8: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 9: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 10: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 11: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 12: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_

# Starting to test for the Output (manually)

In [52]:
def new_func(dict_df_comps):
    '''
    docstring
    '''

    # For Loop for each compartment (compartment-level)
    for key, value in slicing_dict.items():
        #print(key)
        #print(value)
        compartment_num = key
        print(key, f'compartment')
        dict_df_comps[key] = data_frame.copy()

        # For Loop for each bounding box
        for k, v in cp_bb.items():

            # calling function: checking_bounding_box (Output = True or False)
            x_min = v[0]
            y_min = v[1]
            x_max = v[2]
            y_max = v[3]
            class_object = v[4]
            #print(x_min)
            #print(y_min)
            #print(x_max)
            #print(y_max)
            if checking_bounding_box(compartment_num):
                print('True')
                #calling the function: checking_for_9_cases
                x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
                # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
                abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
                # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

                #Adding those to a dataframe with the original class
                list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
                dict_df_comps[key].loc[k] = list_ready
                # else:
                # print('False')
                # continue or cut the else
        # print(dict_df_comps[key])
    return dict_df_comps

        #Output is the dict_df_comps

In [53]:
data_frame, dict_df_comps = preparing_for_for_loops()

dont_care = new_func(dict_df_comps)
dont_care

0 compartment
1 compartment
2 compartment
3 compartment
4 compartment
5 compartment
6 compartment
7 compartment
8 compartment
9 compartment
10 compartment
11 compartment
12 compartment
13 compartment
14 compartment
15 compartment
16 compartment
17 compartment
18 compartment
19 compartment
20 compartment
21 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
22 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
23 compartment
24 compartment
25 compartment
26 compartment
27 compartment
28 compartment
29 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
30 compartment
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
31 compartment
32 compartment
33 compartment
34 compartment
35 compartment
36 compartment
37 compartment
38 compartment
39 compartment
40 compartment
41 compartment
42 compartment
43 compartment
44 compartment
45 compartment


{0: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 1: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 2: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 3: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 4: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 5: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 6: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 7: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 8: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 9: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 10: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 11: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_max]
 Index: [],
 12: Empty DataFrame
 Columns: [class, x_min, y_min, x_max, y_

In [14]:
# Create an empty DataFrame with the specified columns
columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
data_frame = pd.DataFrame(columns=columns)
data_frame

# Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
dict_df_comps = {}

for i in range (0, 48):
  dict_df_comps[i] = f'df_comp_{i}'

# For Loop for each compartment (compartment-level)
for key, value in slicing_dict.items():
  #print(key)
  #print(value)
  compartment_num = key
  print(key, f'compartment')
  dict_df_comps[key] = data_frame.copy()

  # For Loop for each bounding box
  for k, v in cp_bb.items():

    # calling function: checking_bounding_box (Output = True or False)
    x_min = v[0]
    y_min = v[1]
    x_max = v[2]
    y_max = v[3]
    class_object = v[4]
    #print(x_min)
    #print(y_min)
    #print(x_max)
    #print(y_max)
    if checking_bounding_box(compartment_num):
      print('True')
      #calling the function: checking_for_9_cases
      x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp = checking_for_9_cases()
      # print(x_min_tmp, y_min_tmp, x_max_tmp, y_max_tmp)
      abs_x_min, abs_y_min, abs_x_max, abs_y_max = to_absolute_coordinates_xyxy()
      # print(abs_x_min, abs_y_min, abs_x_max, abs_y_max)

      #Adding those to a dataframe with the original class
      list_ready = [class_object, abs_x_min, abs_y_min, abs_x_max, abs_y_max]
      dict_df_comps[key].loc[0] = list_ready
    # else:
      # print('False')
      # continue or cut the else
  print(dict_df_comps[key])

  #Output is the dict_df_comps

0 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
1 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
2 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
3 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
4 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
5 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
6 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
7 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
8 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
9 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
10 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_max, y_max]
Index: []
11 compartment
Empty DataFrame
Columns: [class, x_min, y_min, x_

,class,x_min,y_min,x_max,y_max
0,0,135,221,327,590
